# Intro to Data

The data used in this analysis was sourced from the California Department of Insurance. It includes key variables relevant to evaluating homeowners insurance trends and risks at the ZIP code level.

Two sheets of the same Excel file are used:

2020HO – Contains ZIP-level data for 2020, including earned premiums, catastrophic losses, number of dwellings affected, and other exposure-related metrics.

2021HO – Contains ZIP-level data for 2021, including earned premiums and number of dwellings affected.

In [143]:
import numpy as np 
import pandas as pd 



# Data Ingestion

In [151]:
file_path = '/Users/shireen/Documents/CaliforniaWildfireAnalysis/data/wildfire_loss_and_premium_data.xlsx'
sheets_to_load = ["2020HO", "2021HO"]
data_frames ={}
for sheet_name in sheets_to_load:
    df = pd.read_excel(file_path, sheet_name = sheet_name)
    data_frames[sheet_name] = df


features = data_frames["2020HO"]
labels = data_frames['2021HO']

print(features.head())
print(labels.head())

print(f"features Shape : {features.shape}")
print(f"labels Shape : {labels.shape}")


   ZIP Code  Cov A Amount Weighted Avg  Cov C Amount Weighted Avg  \
0     90001                  261658.94                  174339.07   
1     90002                  250804.27                  167880.40   
2     90003                  266381.48                  177789.87   
3     90004                  929430.46                  635509.87   
4     90005                 1329088.35                  852256.91   

   Avg Fire Risk Score  Number of Negligible Fire Risk Exposure  \
0                 0.32                                      884   
1                 0.33                                     1270   
2                 0.31                                     1195   
3                 0.45                                      962   
4                 0.44                                      224   

   Number of Low Fire Risk Exposure  Number of Moderate Fire Risk Exposure  \
0                               407                                      0   
1                         

# Data Profiling

In [152]:
# 1. Remove the new line from the headers
# 2. Check for NA's in the features as well as the labels
# 3. Check the column data types, change it if neccessary
# 4. Also, check for distinct values

def clean_col_names(df):
    cols = df.columns
    new_cols = []
    for col in cols:
        # Replace newlines, strip leading/trailing whitespace
        new_col = col.replace('\n', ' ').strip()
        # Replace multiple spaces with single space
        new_col = ' '.join(new_col.split())
        new_cols.append(new_col)
    df.columns = new_cols
    return df

features = clean_col_names(features)
labels = clean_col_names(labels)

print(features.head())
print(labels.head())

   ZIP Code  Cov A Amount Weighted Avg  Cov C Amount Weighted Avg  \
0     90001                  261658.94                  174339.07   
1     90002                  250804.27                  167880.40   
2     90003                  266381.48                  177789.87   
3     90004                  929430.46                  635509.87   
4     90005                 1329088.35                  852256.91   

   Avg Fire Risk Score  Number of Negligible Fire Risk Exposure  \
0                 0.32                                      884   
1                 0.33                                     1270   
2                 0.31                                     1195   
3                 0.45                                      962   
4                 0.44                                      224   

   Number of Low Fire Risk Exposure  Number of Moderate Fire Risk Exposure  \
0                               407                                      0   
1                         

In [153]:
def column_summary(df):
    summary_data = []
    
    for col_name in df.columns:
        col_dtype = df[col_name].dtype
        num_of_nulls = df[col_name].isnull().sum()
        num_of_non_nulls = df[col_name].notnull().sum()
        num_of_distinct_values = df[col_name].nunique()
        
        if num_of_distinct_values <= 10:
            distinct_values_counts = df[col_name].value_counts().to_dict()
        else:
            top_10_values_counts = df[col_name].value_counts().head(10).to_dict()
            distinct_values_counts = {k: v for k, v in sorted(top_10_values_counts.items(), key=lambda item: item[1], reverse=True)}

        summary_data.append({
            'col_name': col_name,
            'col_dtype': col_dtype,
            'num_of_nulls': num_of_nulls,
            'num_of_non_nulls': num_of_non_nulls,
            'num_of_distinct_values': num_of_distinct_values,
            'distinct_values_counts': distinct_values_counts
        })
    
    summary_df = pd.DataFrame(summary_data)
    return summary_df

# featurers of 2020 dataset
summary_df = column_summary(features)
display(summary_df)
features.describe()

,col_name,col_dtype,num_of_nulls,num_of_non_nulls,num_of_distinct_values,distinct_values_counts
0,ZIP Code,int64,0,2118,2118,"{90001: 1, 94954: 1, 94979: 1, 94978: 1, 94974..."
1,Cov A Amount Weighted Avg,float64,0,2118,1849,"{0.0: 267, 362000.0: 2, 547000.0: 2, 1100000.0..."
2,Cov C Amount Weighted Avg,float64,0,2118,1847,"{0.0: 270, 649500.0: 2, 550000.0: 2, 174339.07..."
3,Avg Fire Risk Score,float64,0,2118,303,"{0.0: 321, 0.34: 45, 0.31: 44, 0.3: 38, 0.32: ..."
4,Number of Negligible Fire Risk Exposure,int64,0,2118,1050,"{0: 458, 1: 119, 2: 64, 3: 33, 5: 28, 4: 26, 6..."
5,Number of Low Fire Risk Exposure,int64,0,2118,1019,"{0: 426, 1: 59, 2: 30, 4: 24, 3: 18, 7: 15, 11..."
6,Number of Moderate Fire Risk Exposure,int64,0,2118,514,"{0: 636, 1: 136, 2: 67, 3: 47, 4: 42, 5: 29, 9..."
7,Number of High Fire Risk Exposure,int64,0,2118,396,"{0: 977, 1: 112, 2: 42, 6: 24, 4: 23, 3: 17, 8..."
8,Number of Very High Fire Risk Exposure,int64,0,2118,133,"{0: 1292, 1: 136, 2: 76, 3: 64, 4: 41, 5: 39, ..."
9,Earned Premium,int64,0,2118,1845,"{0: 270, 109107: 2, 2517: 2, 540: 2, 129: 2, 8..."


,ZIP Code,Cov A Amount Weighted Avg,Cov C Amount Weighted Avg,Avg Fire Risk Score,Number of Negligible Fire Risk Exposure,Number of Low Fire Risk Exposure,Number of Moderate Fire Risk Exposure,Number of High Fire Risk Exposure,Number of Very High Fire Risk Exposure,Earned Premium,...,Non-CAT Cov C Smoke - Incurred Losses,Non-CAT Cov C Smoke - Number of Claims,CAT Cov A Fire - Incurred Losses,CAT Cov A Fire - Number of Claims,CAT Cov A Smoke - Incurred Losses,CAT Cov A Smoke - Number of Claims,CAT Cov C Fire - Incurred Losses,CAT Cov C Fire - Number of Claims,CAT Cov C Smoke - Incurred Losses,CAT Cov C Smoke - Number of Claims
count,2118.000000,2.118000e+03,2.118000e+03,2118.000000,2118.000000,2118.000000,2118.000000,2118.000000,2118.000000,2.118000e+03,...,2118.000000,2118.000000,2.118000e+03,2118.000000,2.118000e+03,2118.000000,2.118000e+03,2118.000000,2.118000e+03,2118.000000
mean,93601.748347,4.463992e+05,2.946678e+05,0.880345,1010.971199,655.754958,143.809726,97.228517,11.024551,2.286151e+06,...,3125.233711,0.359301,5.237118e+05,2.447120,3.473180e+04,2.796506,1.857068e+05,1.777620,6.822640e+03,1.142115
std,1787.321945,4.371201e+05,2.260625e+05,0.870823,1482.279691,860.877615,306.126254,279.619178,39.283959,3.011091e+06,...,18207.476014,1.725845,6.906671e+06,21.678324,2.541915e+05,15.520649,2.410128e+06,18.223515,7.351850e+04,10.832434
min,90001.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,...,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000
25%,92242.250000,2.964608e+05,2.002615e+05,0.290000,1.000000,3.000000,0.000000,0.000000,0.000000,4.141575e+04,...,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000
50%,93607.500000,3.839233e+05,2.646546e+05,0.490000,152.000000,231.500000,11.000000,1.000000,0.000000,9.494070e+05,...,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000
75%,95332.250000,5.196659e+05,3.562095e+05,1.340000,1665.250000,1092.750000,130.000000,62.000000,4.000000,3.720230e+06,...,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000
max,96162.000000,1.029831e+07,3.089494e+06,4.000000,9762.000000,9045.000000,4595.000000,4224.000000,806.000000,3.315174e+07,...,429669.000000,30.000000,1.429249e+08,562.000000,7.458821e+06,319.000000,4.979913e+07,472.000000,2.103006e+06,363.000000


In [154]:
features.duplicated().sum()

0

Observation:

1. The dataset appears to be clean, with no missing values.

2. ZIP code column contains 2,118 unique ZIP codes, matching the total number of rows.

3. The Average Fire Risk Score ranges from 0 to 4.0, with a mean of 0.88. This indicates that the values are continuous and not restricted to integers.
4. No duplicate rows.

Some features contain extremely high values, suggesting the presence of potential outliers.

Notes:

Ensure that all ZIP codes from the 2020 dataset are also present in the 2021 dataset along with their premium data.

Investigate and handle potential outliers to avoid skewing the analysis.

In [147]:
# labels of 2021 dataset
summary_df = column_summary(labels)
display(summary_df)
labels.describe()

,col_name,col_dtype,num_of_nulls,num_of_non_nulls,num_of_distinct_values,distinct_values_counts
0,ZIP Code,int64,0,2118,2118,"{90001: 1, 94954: 1, 94979: 1, 94978: 1, 94974..."
1,Cov A Amount Weighted Avg,float64,0,2118,1829,"{0.0: 289, 500000.0: 2, 278650.33: 1, 522459.8..."
2,Cov C Amount Weighted Avg,float64,0,2118,1826,"{0.0: 293, 183442.31: 1, 355663.33: 1, 359887...."
3,Avg Fire Risk Score,float64,0,2118,304,"{0.0: 330, 0.41: 36, 0.34: 36, 0.36: 36, 1.0: ..."
4,Number of Negligible Fire Risk Exposure,int64,0,2118,1028,"{0: 537, 1: 118, 2: 50, 3: 28, 4: 21, 6: 16, 7..."
5,Number of Low Fire Risk Exposure,int64,0,2118,1059,"{0: 424, 1: 61, 2: 24, 5: 19, 3: 19, 12: 16, 4..."
6,Number of Moderate Fire Risk Exposure,int64,0,2118,551,"{0: 561, 1: 112, 2: 59, 3: 49, 4: 37, 6: 30, 5..."
7,Number of High Fire Risk Exposure,int64,0,2118,397,"{0: 945, 1: 129, 2: 35, 3: 32, 5: 27, 4: 23, 9..."
8,Number of Very High Fire Risk Exposure,int64,0,2118,131,"{0: 1271, 1: 134, 2: 79, 3: 56, 4: 43, 6: 34, ..."
9,Earned Premium,int64,0,2118,1825,"{0: 291, 1016: 2, 25814: 2, 1400: 2, 1076066: ..."


,ZIP Code,Cov A Amount Weighted Avg,Cov C Amount Weighted Avg,Avg Fire Risk Score,Number of Negligible Fire Risk Exposure,Number of Low Fire Risk Exposure,Number of Moderate Fire Risk Exposure,Number of High Fire Risk Exposure,Number of Very High Fire Risk Exposure,Earned Premium,...,Non-CAT Cov C Smoke - Incurred Losses,Non-CAT Cov C Smoke - Number of Claims,CAT Cov A Fire - Incurred Losses,CAT Cov A Fire - Number of Claims,CAT Cov A Smoke - Incurred Losses,CAT Cov A Smoke - Number of Claims,CAT Cov C Fire - Incurred Losses,CAT Cov C Fire - Number of Claims,CAT Cov C Smoke - Incurred Losses,CAT Cov C Smoke - Number of Claims
count,2118.000000,2.118000e+03,2.118000e+03,2118.000000,2118.000000,2118.000000,2118.000000,2118.000000,2118.000000,2.118000e+03,...,2118.000000,2118.000000,2.118000e+03,2118.000000,2.118000e+03,2118.000000,2.118000e+03,2118.000000,2118.000000,2118.000000
mean,93601.748347,4.723614e+05,3.079263e+05,0.901449,947.142115,710.681775,169.560434,89.990085,11.421152,2.548583e+06,...,2294.961284,0.224740,6.419612e+04,0.438149,5.671970e+03,0.461284,2.904912e+04,0.364023,1098.900378,0.128895
std,1787.321945,4.506763e+05,2.355994e+05,0.858683,1401.291991,923.899696,340.192221,255.789473,41.856517,3.368130e+06,...,14665.300899,1.230809,1.370309e+06,5.389829,4.876728e+04,2.705016,6.379906e+05,4.942195,20419.010567,1.324849
min,90001.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,...,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000
25%,92242.250000,3.140164e+05,2.090431e+05,0.330000,0.000000,4.000000,0.000000,0.000000,0.000000,4.834300e+04,...,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000
50%,93607.500000,4.101377e+05,2.784076e+05,0.550000,133.500000,245.500000,17.000000,1.000000,0.000000,1.050004e+06,...,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000
75%,95332.250000,5.541236e+05,3.760620e+05,1.340000,1540.000000,1194.500000,176.250000,60.750000,5.000000,4.116438e+06,...,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000
max,96162.000000,1.029831e+07,3.089494e+06,4.000000,9828.000000,8799.000000,4697.000000,3833.000000,991.000000,3.627053e+07,...,300000.000000,26.000000,4.990782e+07,162.000000,1.415438e+06,69.000000,2.445256e+07,163.000000,765588.000000,39.000000


In [155]:
# We need to make sure, we have 2021 premium amount for all the zipcodes of 2020

features_zip_code = set(features["ZIP Code"])
labels_zip_code = set(labels["ZIP Code"])

for zip_code_2021 in labels_zip_code:
    if(zip_code_2021 not in features_zip_code):
        print(f"zip code not available: {zip_code_2021} ")


Observation:

The 2021 dataset is also clean, containing no missing values and exactly 2,118 rows.

The ZIP Code column has 2,118 unique entries, indicating no duplicates. Also, all the zipcodes that are present in the 2020 data are also present in 2021

# Data Pre-processing Pipeline

The dataset is already well-structured and clean, requiring minimal preprocessing. However, I made a small adjustment by __removing newline characters__ from column headers to improve accessibility and usability.

Since the analysis focuses on understanding how catastrophic losses in 2020 influenced premium rates in 2021, I will:

* Merge the 2020 and 2021 datasets into a single DataFrame based on ZIP codes.

* Exclude details related to 2021 catastrophic losses, as they are not relevant to the prediction objective.



In [163]:
def cleaning_pipeline(features : pd.DataFrame, labels: pd.DataFrame, path: str):
    labels=labels.rename(columns = {"Earned Premium": "Earned Premium 2021", "Earned Exposure":"Earned Exposure 2021", "ZIP Code": "ZIP Code"})
    labels=labels[["Earned Premium 2021", "Earned Exposure 2021", "ZIP Code"]]
    features = features.rename(columns = {"Earned Premium": "Earned Premium 2020", "Earned Exposure": "Earned Exposure 2020"})
    features = pd.merge(features, labels, on ="ZIP Code" , how ='inner')
    features.to_csv(path, index=False)

cleaning_pipeline(features, labels,'/Users/shireen/Documents/CaliforniaWildfireAnalysis/data/supervised_clean_data.csv')
